In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import yaml
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, r
import rpy2.robjects as robjects
import json

def linearRegressionScore(X, y):
    # using linear regression score function to compute the r2
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X, y)
    r_squared = model.score(X, y)
    return r_squared

def LR(data):
    ### normalize the confounders, where u is the mean of the training samples
    dataCopy = data.copy()
    try:
        dataCopyX = dataCopy.drop(columns = ["prediction", "label"])
    except:
        dataCopyX = dataCopy.drop(columns = ["prediction"])
    model = make_pipeline(StandardScaler(), LinearRegression())
    # print(dataCopyX.columns)
    residual = model.fit(dataCopyX, dataCopy["prediction"])
    dataCopy["LRPrediction"] = residual.predict(dataCopyX)
    return dataCopy 

def LRWithCol(data, col):
    ### normalize the confounders, where u is the mean of the training samples
    dataCopy = data.copy()
    dataCopyX = dataCopy.drop(columns = [col])
    model = make_pipeline(StandardScaler(), LinearRegression())
    # print(dataCopyX.columns)
    residual = model.fit(dataCopyX, dataCopy[col])
    dataCopy["LRPrediction"] = residual.predict(dataCopyX)
    return dataCopy 

def evaluation(predictionAge):
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mean_squared_error = mean_squared_error(predictionAge["Age"].values, predictionAge["prediction"].values)
    mean_absolute_error = mean_absolute_error(predictionAge["Age"].values, predictionAge["prediction"].values)
    r2_score = linearRegressionScore(predictionAge[["Age"]], predictionAge[["prediction"]])
    dataCorr = predictionAge.drop(columns=["Age"]).corrwith(predictionAge['Age'], method='pearson')
    return mean_squared_error, mean_absolute_error, r2_score, dataCorr.values[0]

def AgeAccelerationResidual(
 control,
 case,
 method, #AA or IEAA or Basic or Full 
):
    model = make_pipeline(StandardScaler(), LinearRegression())
    control = control.astype(float)
    case = case.astype(float)
    try:
        dataX = control.drop(columns = ["prediction", "label", "LRPrediction"])
        caseX = case.drop(columns = ["prediction", "label", "LRPrediction"])
    except:
        dataX = control.drop(columns = ["prediction", "LRPrediction"])
        caseX = case.drop(columns = ["prediction", "LRPrediction"])  
    print(dataX.columns)
    controlModel = model.fit(dataX, control["prediction"])
    case[method] = case["prediction"] - controlModel.predict(caseX) 
    control[method] = control["prediction"] - control["LRPrediction"]
    return control, case

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
5: Setting LC_PAPER failed, using "C" 
6: Setting LC_MEASUREMENT failed, using "C" 


In [6]:
with open('./config.yml', 'r') as file:
     root = yaml.safe_load(file)

# discovery3KPredictionAge contains the biologically predicted age computed by 'pathwayAge'
analysisPath = root["pathway"]["analysisData"]
predictionResult = analysisPath.format("discovery3KPredictionAge.csv")
predictionResult = pd.read_csv(predictionResult, index_col=0)

confunder = analysisPath.format("discovery3KCovariateData.csv")
confunder = pd.read_csv(confunder, index_col="Sample")

label = confunder[["Label"]].astype(int)
prediction = label.join(predictionResult)
controlOriginal = prediction[prediction.Label.eq(0)].drop(columns="Label")
caseOriginal = prediction[prediction.Label.eq(1)].drop(columns="Label")
controlOriginal
# caseOriginal.shape

,prediction,Age
Sample,,
GSM2333901,75.327036,72.0
GSM2333902,56.469699,55.0
GSM2333903,22.008102,23.0
GSM2333904,91.454184,86.0
GSM2333905,71.997623,74.0
...,...,...
7796814021_R04C02,54.317598,52.7
7796814021_R05C01,46.786595,44.3
7796814021_R05C02,46.714393,46.4


In [7]:
#  Evaluation  

mean_squared_error, mean_absolute_error, r2_score, dataCorr = evaluation(controlOriginal)
data = [mean_squared_error,mean_absolute_error,r2_score, dataCorr]

Control = pd.DataFrame(data=data, columns = ["Discovery3K"], index=["MSE", "MAE", "R2", "Rho"])
Control["Tag"] = "Control" 
Control

,Discovery3K,Tag
MSE,9.805735,Control
MAE,2.345776,Control
R2,0.944302,Control
Rho,0.971752,Control


In [8]:
# acceleration residual

# a. aa: no adjust
# b. basic: sex and cohort adjusted
# c. IEAA:  intrinsic epigenetic age acceleration - blood cell composition adjusted
# d. Full: sex, blood cell composition, population and smoking adjusted

confunder = confunder.drop(columns = ["Age", "Label"])
Full = confunder
basic = confunder[["Female", "CohortTag"]]
cellComposition = ['CD8.naive', 'CD8pCD28nCD45RAn', 'PlasmaBlast', 'CD4T', 'NK', 'Mono', 'Gran', 'CohortTag']
cellCount = confunder[cellComposition]

controlAA = LR(controlOriginal)
caseAA = controlAA

controlCellCount = controlOriginal.join(cellCount)
controlIEAA = LR(controlCellCount)
caseIEAA = controlIEAA

controlBasic = controlOriginal.join(basic)
controlBasic = LR(controlBasic)
caseBasic = controlBasic

controlFull = controlOriginal.join(Full)
controlFull = LR(controlFull)
caseFull = controlFull


controlAA, caseAA = AgeAccelerationResidual(controlAA, caseAA, "AA")
controlBasic, caseBasic = AgeAccelerationResidual(controlBasic, caseBasic, "Basic")
controlIEAA, caseIEAA = AgeAccelerationResidual(controlIEAA, caseIEAA, "IEAA")
controlFull, caseFull = AgeAccelerationResidual(controlFull, caseFull, "Full")

controlAA["Basic"] = controlBasic["Basic"]
caseAA["Basic"] = caseBasic["Basic"]
controlAA["IEAA"] = controlIEAA["IEAA"]
caseAA["IEAA"] = caseIEAA["IEAA"]
controlAA["Full"] = controlFull["Full"]
caseAA["Full"] = caseFull["Full"]

caseAA["Tag"] = "Case"
controlAA["Tag"] = "Control"

ageGapFullAdjusted = controlAA
ageGapFullAdjusted

Index(['Age'], dtype='object')
Index(['Age', 'Female', 'CohortTag'], dtype='object')
Index(['Age', 'CD8.naive', 'CD8pCD28nCD45RAn', 'PlasmaBlast', 'CD4T', 'NK',
       'Mono', 'Gran', 'CohortTag'],
      dtype='object')
Index(['Age', 'population_0', 'population_1', 'population_2', 'population_3',
       'population_4', 'population_5', 'population_6', 'population_7',
       'population_8', 'population_9', 'smoking', 'Female', 'CD8.naive',
       'CD8pCD28nCD45RAn', 'PlasmaBlast', 'CD4T', 'NK', 'Mono', 'Gran',
       'CohortTag'],
      dtype='object')


,prediction,Age,LRPrediction,AA,Basic,IEAA,Full,Tag
Sample,,,,,,,,
GSM2333901,75.327036,72.0,70.872567,4.454469,4.617835,5.733912,4.800795,Control
GSM2333902,56.469699,55.0,54.761096,1.708603,1.856677,1.720638,3.354750,Control
GSM2333903,22.008102,23.0,24.433621,-2.425519,-2.306230,-1.357729,-0.571406,Control
GSM2333904,91.454184,86.0,84.140837,7.313346,7.489305,8.152758,6.978977,Control
GSM2333905,71.997623,74.0,72.768034,-0.770411,-0.605247,0.137977,1.751358,Control
...,...,...,...,...,...,...,...,...
7796814021_R04C02,54.317598,52.7,52.581309,1.736289,1.637582,2.378305,1.898388,Control
7796814021_R05C01,46.786595,44.3,44.620346,2.166248,2.059985,2.294047,1.950760,Control
7796814021_R05C02,46.714393,46.4,46.610587,0.103807,0.131095,-0.798582,0.223419,Control


In [12]:
# acceleration residual per GO  

# To optimize RAM usage, it's worth noting that 'Discovery3KData4Stage2Sub' 
# serves as a subspace strictly for demonstration purposes

AgeGapList = []
data4Stage2 = analysisPath.format('discovery3KData4Stage2Sub.csv')
data4Stage2 = pd.read_csv(data4Stage2, index_col=0)   
data4Stage2 = data4Stage2.drop(columns= ["Age"])   

for col in data4Stage2.columns:
    Full = confunder
    controlFull = data4Stage2[[col]].join(Full).dropna()   
    controlFull = LRWithCol(controlFull, col)
    model = make_pipeline(StandardScaler(), LinearRegression())
    controlFull = controlFull.astype(float)
    dataX = controlFull.drop(columns = [col, "LRPrediction"])
    controlModel = model.fit(dataX, controlFull[col])
    controlFull["Full"] = controlFull[col] - controlFull["LRPrediction"]
    controlFull = controlFull[["Full"]].rename(columns={"Full": col})
    AgeGapList.append(controlFull)
AgeGapGO = pd.concat(AgeGapList, axis=1)
# save for WGCNA in GO level
AgeGapGO.to_csv("./discovery3KAgeGapPerGOSub.csv")
AgeGapGO

,GO:0000002,GO:0000012,GO:0000027,GO:0000028,GO:0000038,GO:0000045,GO:0000050,GO:0000070,GO:0000076,GO:0000077,...,GO:0007007,GO:0007009,GO:0007010,GO:0007015,GO:0007017,GO:0007018,GO:0007019,GO:0007020,GO:0007026,GO:0007029
7786915049_R03C02,-0.315760,-0.414192,-0.499176,-0.044979,-0.248418,-0.226601,0.185669,0.003157,0.068014,-0.202367,...,0.012278,0.068038,-0.054381,0.016962,-0.369048,-0.071130,-0.112018,-0.126205,0.178809,-0.070714
7471147041_R05C01,-0.134717,0.450316,0.110974,0.205162,-0.463183,-0.275722,-0.177040,-0.095787,-0.053217,-0.162023,...,-0.086248,0.107537,0.001381,-0.068483,0.140022,0.042536,0.108605,-0.062045,0.041811,-0.289862
7507867089_R02C02,-0.079746,-0.342584,-0.106064,-0.060620,0.296908,-0.116394,-0.325199,-0.165136,-0.283866,-0.316336,...,0.240043,0.096393,-0.083402,0.056051,0.120082,-0.146851,-0.160353,-0.166621,-0.148934,0.102591
GSM2334126,0.132386,0.334782,-0.500457,-0.160537,0.051313,-0.035228,0.241233,-0.159339,-0.367243,-0.135784,...,0.097332,0.226516,-0.025017,0.182435,0.103693,-0.104238,0.015439,0.307583,-0.396325,0.049272
7471147057_R04C02,0.039565,-0.053617,0.077791,0.160774,-0.006174,0.031015,-0.049635,0.192759,-0.027911,0.292806,...,-0.089867,0.213920,0.196735,0.185474,0.215091,0.147397,0.421265,0.255318,-0.268979,-0.009825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2334536,-0.381842,0.366795,-0.341318,-0.197039,-0.010660,0.221726,0.031290,0.021817,0.102287,0.146252,...,0.212685,-0.028441,-0.039549,0.019083,-0.245269,0.204971,-0.445888,0.330398,0.140039,0.166402
7471147070_R05C01,0.196131,0.136143,-0.247151,-0.140404,-0.052701,-0.168535,-0.250778,-0.010409,0.070684,0.360805,...,-0.080790,-0.508762,-0.049476,0.042890,-0.075016,-0.042861,-0.184054,-0.046321,0.431349,-0.242872
7471147138_R03C02,-0.109589,-0.253925,0.083106,-0.177697,-0.046624,-0.209269,-0.245175,0.289464,-0.048622,-0.001079,...,-0.155289,-0.667110,-0.149721,-0.072851,-0.165428,-0.139230,0.336571,-0.205077,0.211425,-0.113636
7471147095_R06C02,-0.012277,0.039464,0.091969,0.191080,-0.368598,-0.027392,-0.251460,0.208459,0.169439,-0.015474,...,-0.068428,0.175479,0.157315,0.288501,-0.238351,-0.214788,-0.397696,-0.127260,-0.189098,0.049667


In [18]:
# acceleration residual VS GO  
# correlation between each pathway and ageing rate 

control = ageGapFullAdjusted[ageGapFullAdjusted["Tag"].eq("Control")][["Full"]].join(AgeGapGO)
correlationControl = control.drop(columns= ["Full"]).corrwith(control["Full"])
correlationControl = pd.DataFrame(data=correlationControl.values, 
                                  index=correlationControl.index,
                                  columns=["Rho"])

readRDS = r['readRDS']
goDescription = readRDS(analysisPath.format('GO_Description.rds'))
goDescription = dict(zip(goDescription.names, map(list,list(goDescription))))
correlationControl["Description"] = [goDescription[keys][0] for keys in correlationControl.index]
#check the correlation -+ 
correlationControl['Direction'] = np.where(correlationControl["Rho"]<0, "Negtive", "Positive")
correlationControl["RhoAbs"] = correlationControl["Rho"].abs()
correlationControl = correlationControl.sort_values(by=["RhoAbs"], ascending=False)
correlationControl.nlargest(20,"RhoAbs")
# correlationControl[correlationControl.Direction.eq("Positive")].describe()
# correlationControl[correlationControl.Direction.eq("Negtive")].describe() 

,Rho,Description,Direction,RhoAbs
GO:0001649,0.360402,osteoblast differentiation,Positive,0.360402
GO:0006636,0.341099,unsaturated fatty acid biosynthetic process,Positive,0.341099
GO:0001701,0.340370,in utero embryonic development,Positive,0.340370
GO:0001501,0.316049,skeletal system development,Positive,0.316049
GO:0001666,0.304731,response to hypoxia,Positive,0.304731
GO:0006325,0.301048,chromatin organization,Positive,0.301048
GO:0006914,0.285575,autophagy,Positive,0.285575
GO:0006366,0.271741,transcription by RNA polymerase II,Positive,0.271741
GO:0006811,0.271279,ion transport,Positive,0.271279
GO:0000209,0.270048,protein polyubiquitination,Positive,0.270048


In [19]:
# plot   
# Prepare the data for demonstration purposes

metaData = root["pathway"]["metaData"]
golist = metaData.format("golist.json")
with open(golist) as json_file:
    golist = json.load(json_file)

correlationControlTop20 = correlationControl.nlargest(20,"RhoAbs")
correlationControlTop20["GeneCount"] = [len(value) for key, value in golist.items() for index in correlationControlTop20.index if (key == index)]
plotTag = pd.read_excel(analysisPath.format("plotTag.xlsx"), index_col=0)
correlationControlTop20 = correlationControlTop20.join(plotTag)
print(correlationControlTop20)
# correlationControlTop20.to_excel("Temp.xlsx")
coloMap = {
     "developmental process":"#7FB3D5", 
     "metabolic process":"#BC8B56", 
     "response to stimulus": "#F6E758", 
     "cellular process" :"#6D936E",
     "biological regulation" :"#949b9b",
     "localization": "#8c7e78",
     "multicellular organismal process": "#3c4444"
     }

correlationControlTop20["color"] = correlationControlTop20["Tag"].replace(coloMap)
correlationControlTop20 = correlationControlTop20.sort_values(by = ["RhoAbs"], ascending= True)
correlationControlTop20.to_csv("./discovery3KGOSubRankWithGeneCount.csv", index=False)

                 Rho                                        Description  \
GO:0001649  0.360402                         osteoblast differentiation   
GO:0006636  0.341099        unsaturated fatty acid biosynthetic process   
GO:0001701  0.340370                     in utero embryonic development   
GO:0001501  0.316049                        skeletal system development   
GO:0001666  0.304731                                response to hypoxia   
GO:0006325  0.301048                             chromatin organization   
GO:0006914  0.285575                                          autophagy   
GO:0006366  0.271741                 transcription by RNA polymerase II   
GO:0006811  0.271279                                      ion transport   
GO:0000209  0.270048                         protein polyubiquitination   
GO:0007010  0.267150                          cytoskeleton organization   
GO:0006470  0.255721                          protein dephosphorylation   
GO:0001503  0.254766     